In [1]:
from os import path
from pykml import parser
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans


import folium # map rendering library
# Matplotlib and associated plotting modules

import matplotlib.cm as cm
import matplotlib.colors as colors


In [2]:
url      = "https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv"

covid_sp = pd.read_csv(url,encoding='UTF-8', sep=';')

covid_sp.head()

,nome_munic,codigo_ibge,dia,mes,datahora,casos,casos_novos,casos_pc,casos_mm7d,obitos,...,nome_drs,cod_drs,pop,pop_60,area,map_leg,map_leg_s,latitude,longitude,semana_epidem
0,Adamantina,3500105,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,Marília,5,33894,7398,41199,0,8.0,"-21,6820","-51,0737",9
1,Adolfo,3500204,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,São José do Rio Preto,15,3447,761,21106,0,8.0,"-21,2325","-49,6451",9
2,Aguaí,3500303,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,São João da Boa Vista,14,35608,5245,47455,0,8.0,"-22,0572","-46,9735",9
3,Águas da Prata,3500402,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,São João da Boa Vista,14,7797,1729,14267,0,8.0,"-21,9319","-46,7176",9
4,Águas de Lindóia,3500501,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,...,Campinas,3,18374,3275,6013,0,8.0,"-22,4733","-46,6314",9


In [3]:
covid_sp = covid_sp[['nome_munic','dia','mes','obitos','datahora','casos','pop_60','pop','area','latitude','longitude']]

covid_sp.rename(columns={"nome_munic": "name", 
                         "dia": "day",
                         "mes": "month",
                         "obitos": "deaths",
                         "datahora": "date",
                         "casos": "cases"
                        },inplace=True)


covid_sp.head()

,name,day,month,deaths,date,cases,pop_60,pop,area,latitude,longitude
0,Adamantina,25,2,0,2020-02-25,0,7398,33894,41199,"-21,6820","-51,0737"
1,Adolfo,25,2,0,2020-02-25,0,761,3447,21106,"-21,2325","-49,6451"
2,Aguaí,25,2,0,2020-02-25,0,5245,35608,47455,"-22,0572","-46,9735"
3,Águas da Prata,25,2,0,2020-02-25,0,1729,7797,14267,"-21,9319","-46,7176"
4,Águas de Lindóia,25,2,0,2020-02-25,0,3275,18374,6013,"-22,4733","-46,6314"


In [4]:
covid_sp_cases = covid_sp[covid_sp['cases']>0]

covid_sp_cases['date'] = pd.to_datetime(covid_sp_cases['date'])
covid_sp_cases = covid_sp_cases.sort_values(by=['name','date'])
covid_sp_cases['date_diff'] = covid_sp_cases.groupby('name')['date'].diff()/ np.timedelta64(1, 'D')

covid_sp_cases['number_days'] = covid_sp_cases.groupby('name')['date_diff'].cumsum()
covid_sp_cases.head()

C:\Users\jorge\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,name,day,month,deaths,date,cases,pop_60,pop,area,latitude,longitude,date_diff,number_days
26264,Adamantina,14,4,0,2020-04-14,1,7398,33894,41199,"-21,6820","-51,0737",NaN,NaN
26800,Adamantina,15,4,0,2020-04-15,1,7398,33894,41199,"-21,6820","-51,0737",1.0,1.0
27336,Adamantina,16,4,0,2020-04-16,1,7398,33894,41199,"-21,6820","-51,0737",1.0,2.0
27872,Adamantina,17,4,0,2020-04-17,1,7398,33894,41199,"-21,6820","-51,0737",1.0,3.0
28408,Adamantina,18,4,0,2020-04-18,1,7398,33894,41199,"-21,6820","-51,0737",1.0,4.0


In [5]:
covid_sp_cases.head()

,name,day,month,deaths,date,cases,pop_60,pop,area,latitude,longitude,date_diff,number_days
26264,Adamantina,14,4,0,2020-04-14,1,7398,33894,41199,"-21,6820","-51,0737",NaN,NaN
26800,Adamantina,15,4,0,2020-04-15,1,7398,33894,41199,"-21,6820","-51,0737",1.0,1.0
27336,Adamantina,16,4,0,2020-04-16,1,7398,33894,41199,"-21,6820","-51,0737",1.0,2.0
27872,Adamantina,17,4,0,2020-04-17,1,7398,33894,41199,"-21,6820","-51,0737",1.0,3.0
28408,Adamantina,18,4,0,2020-04-18,1,7398,33894,41199,"-21,6820","-51,0737",1.0,4.0


In [19]:
covid_sp_cases_days = covid_sp_cases[covid_sp_cases['number_days']==60.0]
covid_sp_cases_days = covid_sp_cases_days[['name','deaths','cases','pop_60','pop','area','latitude','longitude', 'number_days']].reset_index(drop=True)
covid_sp_cases_days['cases_100M'] = covid_sp_cases_days['cases']/covid_sp_cases_days['pop']*100000
covid_sp_cases_days['IFR'] = covid_sp_cases_days['deaths']/covid_sp_cases_days['cases']
covid_sp_cases_days['pop_den'] = covid_sp_cases_days['pop']/covid_sp_cases_days['area']

In [20]:
covid_sp_cases_days['latitude'] = covid_sp_cases_days['latitude'].str.replace(',','.').astype(float)
covid_sp_cases_days['longitude'] = covid_sp_cases_days['longitude'].str.replace(',','.').astype(float)
covid_sp_cases_days = covid_sp_cases_days[covid_sp_cases_days['name']!="Ignorado"]

covid_sp_cases_days.to_excel("covid_sp_cases_days.xlsx")  

In [21]:
CLIENT_ID = 'W1PSGRUFYBCD4SAU1DFLTF2KRCA4F4GG2MJDCFC1FB10VUKK' # Foursquare ID
CLIENT_SECRET = 'CXGKVFYQUCU3VYWAH41WBUHR0JH2O12NK5WYJX2AEU5ARCJI' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W1PSGRUFYBCD4SAU1DFLTF2KRCA4F4GG2MJDCFC1FB10VUKK
CLIENT_SECRET:CXGKVFYQUCU3VYWAH41WBUHR0JH2O12NK5WYJX2AEU5ARCJI


In [22]:
def getNearbyHospitals(names, latitudes, longitudes, areas, LIMIT=1000):
    
    column_names = ['name', 
                  'number_hospitals']
    
    nearby_hospitals = pd.DataFrame(columns = column_names)
    
    for name, lat, lng, area in zip(names, latitudes, longitudes, areas):
           
        radius = np.sqrt(area * 10000/np.pi)
        categoryId = '4bf58dd8d48988d196941735'
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        new_row = {'name':name, 
                    'number_hospitals':len(results)}
        nearby_hospitals = nearby_hospitals.append(new_row, ignore_index=True)
    
    return(nearby_hospitals)


nearby_hospitals= getNearbyHospitals(names=covid_sp_cases_days['name'],
                                   latitudes=covid_sp_cases_days['latitude'],
                                   longitudes=covid_sp_cases_days['longitude'],
                                    areas = covid_sp_cases_days['area'])
nearby_hospitals

,name,number_hospitals
0,Arujá,4
1,Barueri,8
2,Brodowski,3
3,Caieiras,10
4,Cajamar,1
5,Campinas,96
6,Carapicuíba,9
7,Cotia,16
8,Embu das Artes,0
9,Ferraz de Vasconcelos,4


In [23]:
covid_sp_cases_days_hosp = pd.merge(covid_sp_cases_days,nearby_hospitals,left_on='name',right_on='name',how='left')

In [24]:
covid_sp_cases_days_hosp.head()

,name,deaths,cases,pop_60,pop,area,latitude,longitude,number_days,cases_100M,IFR,pop_den,number_hospitals
0,Arujá,14,194,10776,89744,9617,-23.3965,-46.3200,60.0,216.170440,0.072165,9.331808,4
1,Barueri,82,728,29409,264390,657,-23.5057,-46.8790,60.0,275.350808,0.112637,402.420091,8
2,Brodowski,1,17,3710,23943,27846,-20.9845,-47.6572,60.0,71.001963,0.058824,0.859836,3
3,Caieiras,19,225,12059,100612,9764,-23.3607,-46.7397,60.0,223.631376,0.084444,10.304383,10
4,Cajamar,22,244,7690,77627,13139,-23.3550,-46.8781,60.0,314.323625,0.090164,5.908136,1


In [25]:
covid_sp_cases_days_hosp['pop_60_rate'] = covid_sp_cases_days_hosp['pop_60']/covid_sp_cases_days_hosp['pop']
covid_sp_cases_days_hosp['number_hospitals'] = covid_sp_cases_days_hosp['number_hospitals'].astype(float)
covid_sp_cases_days_hosp.head()

,name,deaths,cases,pop_60,pop,area,latitude,longitude,number_days,cases_100M,IFR,pop_den,number_hospitals,pop_60_rate
0,Arujá,14,194,10776,89744,9617,-23.3965,-46.3200,60.0,216.170440,0.072165,9.331808,4.0,0.120075
1,Barueri,82,728,29409,264390,657,-23.5057,-46.8790,60.0,275.350808,0.112637,402.420091,8.0,0.111233
2,Brodowski,1,17,3710,23943,27846,-20.9845,-47.6572,60.0,71.001963,0.058824,0.859836,3.0,0.154951
3,Caieiras,19,225,12059,100612,9764,-23.3607,-46.7397,60.0,223.631376,0.084444,10.304383,10.0,0.119856
4,Cajamar,22,244,7690,77627,13139,-23.3550,-46.8781,60.0,314.323625,0.090164,5.908136,1.0,0.099063


In [26]:
mod_columns = ['IFR','cases_100M', 'number_hospitals', 'pop_60_rate','pop_den']
covid_sp_cases_days_clustering = covid_sp_cases_days_hosp[mod_columns]
covid_sp_cases_days_clustering.dtypes

IFR                 float64
cases_100M          float64
number_hospitals    float64
pop_60_rate         float64
pop_den             float64
dtype: object

In [41]:
from sklearn.preprocessing import StandardScaler

X = covid_sp_cases_days_clustering.values
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)




from sklearn.metrics import silhouette_score

sil = []
kmax = 10
max_score = 0
k_max_score = 0

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
    kmeans =  KMeans(init="k-means++", n_clusters=k, n_init=12, random_state=12).fit(cluster_dataset)
    labels = kmeans.labels_
    
    sil_score = silhouette_score(cluster_dataset, labels, metric = 'euclidean')
    print(k,sil_score)
    if sil_score > max_score:
        max_score = sil_score
        k_max_score = k
    
num_clusters = k_max_score

print(num_clusters)
k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12, random_state=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

2 0.2698472755691015
3 0.3159805200617762
4 0.3075615194791826
5 0.27926465773516074
6 0.29375580461484335
7 0.3043420058539099
8 0.2777079271576031
9 0.25468343297378493
10 0.23857272740203406
3


In [43]:
covid_sp_cases_days_hosp.drop(['Cluster Labels'], axis=1, inplace=True,  errors='ignore')
covid_sp_cases_days_hosp.insert(0, 'Cluster Labels', labels)

In [44]:
covid_sp_cases_days_hosp.head()

,Cluster Labels,name,deaths,cases,pop_60,pop,area,latitude,longitude,number_days,cases_100M,IFR,pop_den,number_hospitals,pop_60_rate
0,1,Arujá,14,194,10776,89744,9617,-23.3965,-46.3200,60.0,216.170440,0.072165,9.331808,4.0,0.120075
1,2,Barueri,82,728,29409,264390,657,-23.5057,-46.8790,60.0,275.350808,0.112637,402.420091,8.0,0.111233
2,1,Brodowski,1,17,3710,23943,27846,-20.9845,-47.6572,60.0,71.001963,0.058824,0.859836,3.0,0.154951
3,1,Caieiras,19,225,12059,100612,9764,-23.3607,-46.7397,60.0,223.631376,0.084444,10.304383,10.0,0.119856
4,1,Cajamar,22,244,7690,77627,13139,-23.3550,-46.8781,60.0,314.323625,0.090164,5.908136,1.0,0.099063


In [45]:
latitude = -23.5505199
longitude = -46.63330939999997

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(covid_sp_cases_days_hosp['latitude'], covid_sp_cases_days_hosp['longitude'], covid_sp_cases_days_hosp['name'], covid_sp_cases_days_hosp['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:

for k in range(0, num_clusters):
    print(covid_sp_cases_days_hosp.loc[covid_sp_cases_days_hosp['Cluster Labels'] == k, mod_columns].describe().loc[['mean']])

           IFR  cases_100M  number_hospitals  pop_60_rate   pop_den
mean  0.057213  170.206103         58.615385     0.168576  27.89808
           IFR  cases_100M  number_hospitals  pop_60_rate    pop_den
mean  0.074199  141.279834          7.464286     0.127013  31.241086
           IFR  cases_100M  number_hospitals  pop_60_rate     pop_den
mean  0.084574  150.126134          3.666667     0.124647  426.280859
